In [2]:
import time
import os
import pickle
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [5, 5]
plt.ion()
import numpy as np
from sklearn.decomposition import PCA


import torchvision.transforms as transforms
import torchvision
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import bz2

import _pickle as cPickle
import pandas as pd 
import glob

from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneOut
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import random

from torch.nn.utils.rnn import pad_sequence

In [3]:
stimulus_sEEG_data = np.load('/media/data_cifs_lrs/projects/prj_tutorial_seeg-decoding/seeg_contacts.npy')

In [4]:
stimulus_sEEG_data.shape

(84, 7526347)

In [5]:
face_labels=np.load('/media/data_cifs_lrs/projects/prj_tutorial_seeg-decoding/seconds_with_tony.npy')

In [6]:
num_splits =stimulus_sEEG_data.shape[1]//1024

In [7]:
num_splits

7349

In [8]:
truncated_stimulus_sEEG_data= stimulus_sEEG_data[:,:num_splits*1024]

In [9]:
filtered_stimulus_sEEG_data = np.split(truncated_stimulus_sEEG_data,num_splits,axis=1)

In [10]:
face_labels.shape

(7809,)

In [11]:
truncated_face_labels= face_labels[:num_splits]

In [12]:

# # Find the length of the longest trial
# max_length = max(trial.shape[1] for trial in filtered_stimulus_sEEG_data)

# # Pad each trial's data to the max_length
# padded_data = [np.pad(trial, ((0, 0), (0, max_length - trial.shape[1])), mode='constant') for trial in filtered_stimulus_sEEG_data]



# Flatten the padded data
flattened_data = [trial.flatten() for trial in filtered_stimulus_sEEG_data]


# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(flattened_data, truncated_face_labels, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



# Step 2: Train the SVM
svm_model = SVC(kernel='linear')  
svm_model.fit(X_train, y_train)

# Step 3: Evaluate the Model
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.2f}")